In [1]:
import geopandas as gpd
import scipy.stats as stats
import numpy as np
import pandas as pd
import os,sys,glob
from IPython.display import HTML
import xarray as xr
import rioxarray
import rasterio as rio
from tqdm import tqdm
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import config
import gc

In [34]:
drainagearea = pd.read_csv('../../final_results/IWM-Validation-EXP002/rawdata/hydro_signatures/drainage_area.csv')
drainagearea.set_index('GaugeID',inplace=True)
gauge_list = sorted(glob.glob('/home/civil/phd/cez198621/projects/IWM_verification/final_results/IWM-Validation-EXP002/rawdata/gaugestreamflow/merged/all_year/IWM*.csv'))

In [40]:
drainagearea

,Unnamed: 0,Latitude,Longitude,Drainage_Area
GaugeID,,,,
IWM-gauge-0008,0,13.98,75.69,3376.174316
IWM-gauge-0026,1,13.75,76.35,353.983490
IWM-gauge-0032,2,10.92,79.64,5.002418
IWM-gauge-0043,3,18.78,83.50,4481.907227
IWM-gauge-0061,4,11.65,75.76,0.831566
...,...,...,...,...
IWM-gauge-3825,278,21.21,86.12,8494.139648
IWM-gauge-3826,279,21.30,73.03,0.791049
IWM-gauge-3832,280,16.98,81.46,0.812060


In [ ]:
column_list = ['GaugeID','Latitude','Longitude','Mean Annual Flow (m3/s)',
                                  'Mean Annual Monsoon Flow (m3/s)','Low Flow (m3/s)',
                                  'High Flow (m3/s)','Slope of FDC',
                                  'Annual HF Frequency (Days)','Annual LF Frequency (Days)','Drainage Area (km2)']
hydro_sig = pd.DataFrame(columns=column_list).set_index('GaugeID')
for i in tqdm(range (0,len(gauge_list))):
    try:
        filename = os.path.basename(gauge_list[i])
        s_name = filename.rsplit('-', 1)           
        station = pd.read_csv(gauge_list[i])
        station = station.replace(0, np.nan)
        station.dropna(inplace=True)
        streamflow = station[['Date','Streamflow (cumecs)','Streamflow_tavg']].copy()
        streamflow.set_index('Date',inplace=True)
        streamflow.index = pd.to_datetime(streamflow.index)
        monsoon = streamflow[streamflow.index.map(lambda t: t.month in [6,7,8,9])]
        q_low = streamflow['Streamflow (cumecs)'].quantile(0.05)
        q_high = streamflow['Streamflow (cumecs)'].quantile(0.95)
        q_33 = streamflow['Streamflow (cumecs)'].quantile(0.66)
        q_66 = streamflow['Streamflow (cumecs)'].quantile(0.33)
        fdc_slope = (np.log(q_33)-np.log(q_66))/(0.66-0.33)
        q_annual_mean = streamflow['Streamflow (cumecs)'].groupby(pd.Grouper(freq='1Y')).mean().mean()
        q_monsoon_mean = monsoon['Streamflow (cumecs)'].groupby(pd.Grouper(freq='1Y')).mean().mean()
        hq_freq = streamflow['Streamflow (cumecs)'][streamflow['Streamflow (cumecs)'] > 9*(streamflow['Streamflow (cumecs)'].median())].groupby(pd.Grouper(freq='1Y')).count().mean()
        lq_freq = streamflow['Streamflow (cumecs)'][streamflow['Streamflow (cumecs)'] < 0.2*(streamflow['Streamflow (cumecs)'].mean())].groupby(pd.Grouper(freq='1Y')).count().mean()
        lat = drainagearea.loc[s_name[0],'Latitude']
        lon = drainagearea.loc[s_name[0],'Longitude']
        d_area = drainagearea.loc[s_name[0],'Drainage_Area']
        new_row = pd.DataFrame([[s_name[0],lat,lon,q_annual_mean,q_monsoon_mean,q_low,q_high,fdc_slope,hq_freq,lq_freq,d_area]],columns=column_list).set_index('GaugeID')
        hydro_sig = hydro_sig.append(new_row)
    except Exception as e:
        print (e)
        continue

In [42]:
hydro_sig

,Latitude,Longitude,Mean Annual Flow (m3/s),Mean Annual Monsoon Flow (m3/s),Low Flow (m3/s),High Flow (m3/s),Slope of FDC,Annual HF Frequency (Days),Annual LF Frequency (Days),Drainage Area (km2)
GaugeID,,,,,,,,,,
IWM-gauge-0008,13.98,75.69,78.581587,100.324709,8.750,313.9755,3.221119,4.400000,35.666667,3376.174316
IWM-gauge-0026,13.75,76.35,5.690410,3.455235,0.080,32.7580,4.311673,7.680000,32.240000,353.983490
IWM-gauge-0032,10.92,79.64,8.841004,3.867356,0.380,34.0370,2.791488,3.611111,23.444444,5.002418
IWM-gauge-0043,18.78,83.50,91.807741,143.730271,2.170,398.0540,4.445609,14.437500,59.562500,4481.907227
IWM-gauge-0061,11.65,75.76,51.645570,91.810398,1.170,179.4000,4.983839,4.200000,79.733333,0.831566
...,...,...,...,...,...,...,...,...,...,...
IWM-gauge-3825,21.21,86.12,268.876866,391.173908,5.190,899.7400,6.124544,5.232558,63.837209,8494.139648
IWM-gauge-3826,21.30,73.03,349.053343,354.052888,10.975,1820.0000,5.305192,1.576923,10.518519,0.791049
IWM-gauge-3832,16.98,81.46,20.633571,25.206710,1.050,59.5300,3.697698,2.600000,36.000000,0.812060


In [44]:
hydro_sig.to_csv('../../final_results/IWM-Validation-EXP002/rawdata/hydro_signatures/hydro_signatures.csv')